In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install GPUtil
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#!pip install wandb
import json
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import numpy as np
import pandas as pd




import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 12.0 MB/s 
     |████████████████████████████████| 596 kB 57.0 MB/s 
     |████████████████████████████████| 6.6 MB 63.4 MB/s 
     |████████████████████████████████| 101 kB 9.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 12.9 MB/s 
     |████████████████████████████████| 115 kB 55.1 MB/s 
     |████████████████████████████████| 141 kB 54.2 MB/s 
     |████████████████████████████████| 212 kB 45.2 MB/s 
     |████████████████████████████████| 127 kB 61.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully 

In [3]:

def ConvertLabel2ModelLabel(label):
  if label == 'Positive':
    o = 2
  elif label == 'Neutral':
    o = 1
  elif label == 'Negative':
    o = 0
  elif label == 'spam':
    o = 1
  else: 
    print(" error at ConvertLabel2ModelLabel label is " , label)
    o = np.nan
  return o

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

In [4]:
train_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_3.0:4.0:4.0/train_dataset.csv'
valid_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_3.0:4.0:4.0/valid_dataset.csv'
#model_name = "Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment"
#out_dir = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/tomas23_3:4:4/with_high'
#out_dir2 = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/tomas23_3:4:4/with_high_then_low'

import shutil





name_of_text_column = "text"
hyperparameter_tuning = False
reduce_ = False
num_hyper_trails = 100
epoch = 60

df_train = pd.read_csv(train_path)

df_train_high = df_train[df_train['quality']=="high"]
df_train_low = df_train[df_train['quality']=="low"]

df_valid = pd.read_csv(valid_path)

display(df_train)
display(df_valid)
display(df_train_high)
display(df_train_low)

,Unnamed: 0,text,label,source,quality
0,339,КриптоВести BTC 🤘 Blockasset Taps Well-Known ...,Positive,new_turk_low,low
1,747,Full thanks to as they have analysed this Bit...,Neutral,new_turk_high,high
2,301,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",Neutral,self_manual,high
3,1743,$btc btc health check! No custom death cross; ...,Neutral,new_turk_low,low
4,1108,44 🏦FTX | BTC PERP 🦋 If you have trouble imag...,Neutral,new_turk_low,low
...,...,...,...,...,...
2785,683,This will be very useful in the future because...,Positive,new_turk_low,low
2786,1451,"And without Bitcoin Maxies hyping Bitcoin , th...",Positive,new_turk_low,low
2787,1106,11 IST 88.724 37996.605 ₿ 3371120.946 253...,Positive,new_turk_low,low
2788,1787,Bitcoin is Liberation Network.,Positive,new_turk_low,low


,Unnamed: 0,text,label,source,quality
0,71,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
1,266,Typing mistake for Satoshi not bitcoin,Neutral,incomplete_valid,high
2,410,Saitama SaitamaWolfPack SaitamaInu SaitaMask $...,Neutral,incomplete_valid,high
3,333,"100%, the nuclear industry has no idea what is...",Neutral,new_turk_high,high
4,401,airdrop Airdrops Crypto BSC DeFi Polygon Bitco...,Neutral,self_manual,high
...,...,...,...,...,...
1190,171,This is a very good project so don't miss to j...,Positive,new_turk_high,high
1191,319,Whoever didn’t panic sell we made it 🤝🎉 BTC,Positive,new_turk_high,high
1192,896,"To know more about IPOs and Unlisted shares, c...",Neutral,new_turk_high,high
1193,425,Soon to the moon 🌜 Unitycol $Unity Unityprot...,Positive,incomplete_valid,high


,Unnamed: 0,text,label,source,quality
1,747,Full thanks to as they have analysed this Bit...,Neutral,new_turk_high,high
2,301,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",Neutral,self_manual,high
6,758,Full thanks to as they have analysed this Bit...,Neutral,new_turk_high,high
15,285,BTC DOMINANCE BULLISH PENNANT READY FOR BREAKOUT,Positive,manual,high
25,530,Mining Farm Turning Waste Coal Into Bitcoin Ra...,Positive,new_turk_high,high
...,...,...,...,...,...
2762,11,Bitcoin interoperability platform Interlay rai...,Positive,new_turk_high,high
2772,426,🇧🇷 BRL - R$ 0.00238059 🇺🇸 USD - $ 0.00042746 🇪...,Neutral,self_manual,high
2775,652,Current Bitcoin Price is $40714 BTC Crypto,Neutral,new_turk_high,high
2780,140,btc Crypto cryptocurrency Target 1 close. ...,Positive,new_turk_high,high


,Unnamed: 0,text,label,source,quality
0,339,КриптоВести BTC 🤘 Blockasset Taps Well-Known ...,Positive,new_turk_low,low
3,1743,$btc btc health check! No custom death cross; ...,Neutral,new_turk_low,low
4,1108,44 🏦FTX | BTC PERP 🦋 If you have trouble imag...,Neutral,new_turk_low,low
5,103,Bitcoin maxis are born in bear markets,Positive,new_turk_low,low
7,1768,BTC has risen by 9.19% in the last week. The p...,Neutral,new_turk_low,low
...,...,...,...,...,...
2785,683,This will be very useful in the future because...,Positive,new_turk_low,low
2786,1451,"And without Bitcoin Maxies hyping Bitcoin , th...",Positive,new_turk_low,low
2787,1106,11 IST 88.724 37996.605 ₿ 3371120.946 253...,Positive,new_turk_low,low
2788,1787,Bitcoin is Liberation Network.,Positive,new_turk_low,low


In [5]:
df_train['label'] = df_train['label'].apply(ConvertLabel2ModelLabel)
df_valid['label'] = df_valid['label'].apply(ConvertLabel2ModelLabel)

df_train_high['label'] = df_train_high['label'].apply(ConvertLabel2ModelLabel)
df_train_low['label'] = df_train_low['label'].apply(ConvertLabel2ModelLabel)

display(df_train)
display(df_valid)
display(df_train_high)
display(df_train_low)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Unnamed: 0,text,label,source,quality
0,339,КриптоВести BTC 🤘 Blockasset Taps Well-Known ...,2,new_turk_low,low
1,747,Full thanks to as they have analysed this Bit...,1,new_turk_high,high
2,301,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",1,self_manual,high
3,1743,$btc btc health check! No custom death cross; ...,1,new_turk_low,low
4,1108,44 🏦FTX | BTC PERP 🦋 If you have trouble imag...,1,new_turk_low,low
...,...,...,...,...,...
2785,683,This will be very useful in the future because...,2,new_turk_low,low
2786,1451,"And without Bitcoin Maxies hyping Bitcoin , th...",2,new_turk_low,low
2787,1106,11 IST 88.724 37996.605 ₿ 3371120.946 253...,2,new_turk_low,low
2788,1787,Bitcoin is Liberation Network.,2,new_turk_low,low


,Unnamed: 0,text,label,source,quality
0,71,Full thanks to as they have analysed this Bit...,2,incomplete_valid,high
1,266,Typing mistake for Satoshi not bitcoin,1,incomplete_valid,high
2,410,Saitama SaitamaWolfPack SaitamaInu SaitaMask $...,1,incomplete_valid,high
3,333,"100%, the nuclear industry has no idea what is...",1,new_turk_high,high
4,401,airdrop Airdrops Crypto BSC DeFi Polygon Bitco...,1,self_manual,high
...,...,...,...,...,...
1190,171,This is a very good project so don't miss to j...,2,new_turk_high,high
1191,319,Whoever didn’t panic sell we made it 🤝🎉 BTC,2,new_turk_high,high
1192,896,"To know more about IPOs and Unlisted shares, c...",1,new_turk_high,high
1193,425,Soon to the moon 🌜 Unitycol $Unity Unityprot...,2,incomplete_valid,high


,Unnamed: 0,text,label,source,quality
1,747,Full thanks to as they have analysed this Bit...,1,new_turk_high,high
2,301,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",1,self_manual,high
6,758,Full thanks to as they have analysed this Bit...,1,new_turk_high,high
15,285,BTC DOMINANCE BULLISH PENNANT READY FOR BREAKOUT,2,manual,high
25,530,Mining Farm Turning Waste Coal Into Bitcoin Ra...,2,new_turk_high,high
...,...,...,...,...,...
2762,11,Bitcoin interoperability platform Interlay rai...,2,new_turk_high,high
2772,426,🇧🇷 BRL - R$ 0.00238059 🇺🇸 USD - $ 0.00042746 🇪...,1,self_manual,high
2775,652,Current Bitcoin Price is $40714 BTC Crypto,1,new_turk_high,high
2780,140,btc Crypto cryptocurrency Target 1 close. ...,2,new_turk_high,high


,Unnamed: 0,text,label,source,quality
0,339,КриптоВести BTC 🤘 Blockasset Taps Well-Known ...,2,new_turk_low,low
3,1743,$btc btc health check! No custom death cross; ...,1,new_turk_low,low
4,1108,44 🏦FTX | BTC PERP 🦋 If you have trouble imag...,1,new_turk_low,low
5,103,Bitcoin maxis are born in bear markets,2,new_turk_low,low
7,1768,BTC has risen by 9.19% in the last week. The p...,1,new_turk_low,low
...,...,...,...,...,...
2785,683,This will be very useful in the future because...,2,new_turk_low,low
2786,1451,"And without Bitcoin Maxies hyping Bitcoin , th...",2,new_turk_low,low
2787,1106,11 IST 88.724 37996.605 ₿ 3371120.946 253...,2,new_turk_low,low
2788,1787,Bitcoin is Liberation Network.,2,new_turk_low,low


In [6]:
from transformers import Trainer
from transformers import TrainingArguments
#learning_rate = 5e-05
#batch_size = 16
#eval_batch_size = 4
seed = 40
#optimizer = Adam 
#with betas=(0.9,0.999) and epsilon=1e-08
adam_beta1 = 0.9
adam_beta2 =0.999
lr_scheduler_type = "linear"
num_epochs = 15
#args = TrainingArguments("test_trainer",report_to="wandb" ,logging_strategy = "epoch",evaluation_strategy="epoch",learning_rate = learning_rate,num_train_epochs = num_epochs,lr_scheduler_type =lr_scheduler_type, adam_beta1 = adam_beta1,adam_beta2 =adam_beta2  )


def run_trainer(model_name,out_dir,epoch,batch_size,df_train,df_valid,eval_bool):
  df_train = df_train[['text','label']].set_index('text')
  df_valid = df_valid[['text','label']].set_index('text')
  #display(df_train)
  #display(df_valid)

  dataset = ds.dataset(pa.Table.from_pandas(df_valid).to_batches())
  ### convert to Huggingface dataset
  validation_dataset_torch = Dataset(pa.Table.from_pandas(df_valid))

  dataset = ds.dataset(pa.Table.from_pandas(df_train).to_batches())
  ### convert to Huggingface dataset
  training_dataset_torch = Dataset(pa.Table.from_pandas(df_train))

  #print(training_dataset_torch)
  #print(validation_dataset_torch)

  from transformers import AutoTokenizer

  tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)



  def tokenize_function(data):
      return tokenizer(data['text'], padding="max_length", truncation=True,)


  train_dataset = training_dataset_torch.map(tokenize_function, batched=True)
  eval_dataset = validation_dataset_torch.map(tokenize_function, batched=True)
  print(train_dataset)
  print(eval_dataset)

  import numpy as np
  from datasets import load_metric

  metric = load_metric("matthews_correlation")

  def compute_metrics(eval_pred):
      logits, labels = eval_pred
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels)


  from transformers import AutoModelForSequenceClassification

  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
  #model.to(device)
  args = TrainingArguments(
      #'/content/drive/MyDrive/fyp/fyp2/model/model2-supervised/' + f"{model_name}-finetuned-",
      #'/content/' + f"{model_name}-finetuned-",
      out_dir,
      #report_to="wandb",
      overwrite_output_dir = True,
      logging_dir = out_dir,
      evaluation_strategy = "epoch",
      logging_strategy = "epoch",
      #save_strategy = "NO",
      learning_rate=2.6510704963161386e-06,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epoch,
      weight_decay=0.01,
      metric_for_best_model='matthews_correlation',
      save_total_limit = 2,
      save_strategy = "epoch",
      load_best_model_at_end = True,
      seed = 18
      #push_to_hub=True,
  )



  trainer = Trainer(
      model,
      args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )

  #trainer = Trainer(
    # model=model,
      #args=args,
    # train_dataset=train_dataset,
    #  eval_dataset=eval_dataset,
  #   compute_metrics=compute_metrics,
  #)

  
  if eval_bool:
    eval = trainer.evaluate()
    #print("bob")
    #print(eval)
    
    # training
  else:
    train_result = trainer.train() 
    logs = trainer.state.log_history
    # compute train results
    metrics = train_result.metrics
    max_train_samples = len(train_dataset)
    metrics["train_samples"] = min(max_train_samples, len(train_dataset))

    # save train results
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)

    # compute evaluation results
    metrics = trainer.evaluate()
    max_val_samples = len(eval_dataset)
    metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

    # save evaluation results
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)
    import pickle
    save_logs_dir = out_dir
    name_logs = "logs"
    
    my_file = save_logs_dir + "/" + name_logs + ".p"
    print("saving to ",my_file )

    pickle.dump( logs, open( my_file, "wb" )    ) 

    ld = pickle.load( open( my_file, "rb" ) )


    print(logs)
    print(logs[2])
    print(type(logs))

    print(ld)
    print(type(ld))
    print(logs == ld)
    eval = trainer.evaluate()
    
  import torch
  torch.cuda.is_available() 
  from numba import cuda
  del trainer,args
  torch.cuda.empty_cache()
  #trainer.evaluate()
  return eval



In [7]:

  #run_trainer(model_name="/content/temp/Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment/high/checkpoint-102",out_dir = out,epoch =epoch,df_train = df_train_high,df_valid=df_valid,batch_size=batch_size,eval_bool=True)

def run_both_training(model_name,out_dir="/content/drive/MyDrive/fyp/fyp2/final models and datasets/final models",run_low = False,batch_size = 16,df_valid = df_valid):
  print("starting " ,model_name," at" , "run_low is" , run_low)
  out = out_dir + "/" + model_name + "/high"
  epoch = 40
  import os
  if run_low:
    import os
    for root, dirs, files in os.walk(out):
      break
    #find best checkpoint
    max_mcc = -1.1
    for models in dirs:
      if models.startswith('checkpoint-'):
        #print(models)
        eval_ = run_trainer(model_name=out+"/"+models,out_dir = out,epoch =epoch,df_train = df_train_low,df_valid=df_valid,batch_size=batch_size,eval_bool=True)
        new_max = eval_.get("eval_matthews_correlation")
        if  new_max > max_mcc:
          max_mcc = new_max
          best_models = models
        torch.cuda.empty_cache()

    model_name_for_low = out+"/"+best_models
    out = out_dir + "/" + model_name + "/high_then_low-" + best_models
    if not os.path.exists(out):
      
      # Create a new directory because it does not exist 
      os.makedirs(out)
    else:
      shutil.rmtree(out)
    print("running on checkpoint ",model_name_for_low)
    run_trainer(model_name=model_name_for_low,out_dir = out,epoch =epoch,df_train = df_train_low,df_valid=df_valid,batch_size=batch_size,eval_bool=False)
    torch.cuda.empty_cache()
    print(model_name," is completed for low")

  else:
    if not os.path.exists(out):
      
      # Create a new directory because it does not exist 
      os.makedirs(out)
    else:
      shutil.rmtree(out)

    run_trainer(model_name=model_name,out_dir = out,epoch =epoch,df_train = df_train_high,df_valid=df_valid,batch_size=batch_size,eval_bool = False)
    torch.cuda.empty_cache()
    print(model_name," is completed for high")
    #out = out_dir + "/" + model_name + "/high_then_low"
    #run_trainer(model_name=model_name,out_dir= out,epoch=50,df_train = df_train_low,df_valid=df_valid)

#run_both_training(model_name ="Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment" ,out_dir ="/content/temp",run_low = False)
#run_both_training(model_name ="Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment" ,out_dir ="/content/temp",run_low = True)

In [8]:

#run_both_training("Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment")


#run_both_training("cardiffnlp/bertweet-base-sentiment")
#run_both_training("svalabs/twitter-xlm-roberta-bitcoin-sentiment")

#run_both_training("cardiffnlp/twitter-xlm-roberta-base-sentiment")
#run_both_training("amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061")

#run_both_training("finiteautomata/bertweet-base-sentiment-analysis")




In [ ]:
run_both_training("Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment",run_low = True)
run_both_training("cardiffnlp/bertweet-base-sentiment",run_low = True)

#run_both_training("finiteautomata/bertweet-base-sentiment-analysis",run_low = True)
#run_both_training("svalabs/twitter-xlm-roberta-bitcoin-sentiment",run_low = True)

#run_both_training("cardiffnlp/twitter-xlm-roberta-base-sentiment",run_low = True)
#run_both_training("amansolanki/autonlp-Tweet-Sentiment-Extraction-20114061",run_low = True)



starting  Tomas23/twitter-roberta-base-mar2022-finetuned-sentiment  at run_low is True


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 2251
})
Dataset({
    features: ['label', 'text', 'input_ids', 'attention_mask'],
    num_rows: 1195
})


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1195
  Batch size = 16


...

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.997900,0.786849,0.414393
2,0.875600,0.726169,0.455361
3,0.834700,0.758498,0.442749
4,0.794100,0.756889,0.446081
5,0.754300,0.741197,0.458795
6,0.724500,0.758521,0.440014
7,0.697700,0.765263,0.437693
8,0.665200,0.776697,0.445954
9,0.639400,0.828927,0.421898
10,0.607200,0.899607,0.413460


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1195
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/cardiffnlp/bertweet-base-sentiment/high_then_low-checkpoint-306/checkpoint-141
Configuration saved in /content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/cardiffnlp/bertweet-base-sentiment/high_then_low-checkpoint-306/checkpoint-141/config.json
Model weights saved in /content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/cardiffnlp/bertweet-base-sentiment/high_then_low-checkpoint-306/checkpoint-141/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/cardiffnlp

***** train metrics *****
  epoch                    =       40.0
  total_flos               = 22063713GF
  train_loss               =     0.4229
  train_runtime            = 1:48:28.87
  train_samples            =       2251
  train_samples_per_second =     13.833
  train_steps_per_second   =      0.867


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1195
  Batch size = 16


***** eval metrics *****
  epoch                     =       40.0
  eval_loss                 =     0.7412
  eval_matthews_correlation =     0.4588
  eval_runtime              = 0:00:22.55
  eval_samples              =       1195
  eval_samples_per_second   =     52.991
  eval_steps_per_second     =      3.326
saving to  /content/drive/MyDrive/fyp/fyp2/final models and datasets/final models/cardiffnlp/bertweet-base-sentiment/high_then_low-checkpoint-306/logs.p
[{'loss': 0.9979, 'learning_rate': 2.584793733908235e-06, 'epoch': 1.0, 'step': 141}, {'eval_loss': 0.7868486642837524, 'eval_matthews_correlation': 0.41439262424027556, 'eval_runtime': 22.5777, 'eval_samples_per_second': 52.928, 'eval_steps_per_second': 3.322, 'epoch': 1.0, 'step': 141}, {'loss': 0.8756, 'learning_rate': 2.5185169715003317e-06, 'epoch': 2.0, 'step': 282}, {'eval_loss': 0.7261686325073242, 'eval_matthews_correlation': 0.45536111249109534, 'eval_runtime': 22.5887, 'eval_samples_per_second': 52.903, 'eval_steps_per

In [ ]:
!nvidia-smi 

Fri Aug  5 23:29:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    50W / 250W |    731MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!kill -9 -1